In [1]:

import datetime
import json
import openai
import os
import pandas as pd
import re
from tqdm.auto import tqdm
from typing import List, Union
import zipfile

# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
# LLM wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

/Users/alexmiller/Documents/GitHub/sous-chef-app/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [10]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", 'era'], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

In [11]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

In [12]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [13]:
overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "\n\nSet in Victorian England, Tragedy at Sunset on the Beach tells the story of a young couple, Jack and Lillian, and their fateful journey to the beach on a fateful summer day. Lillian, an aspiring opera singer, is enamored with the idea of a romantic seaside picnic and persuades Jack to accompany her. \n\nWhen they arrive at the beach, they are taken aback by the beauty of the setting sun and the serene atmosphere. However, their joy quickly turns to horror when they realize that a shipwreck has occurred near the shore. In the wreckage, they find the body of a young girl, whose identity is a mystery. \n\nJack and Lillian, in a desperate attempt to help, search the shoreline and eventually find a diary belonging to the girl, which reveals her heartbreaking story. She was a young girl from a wealthy family who had been forced into an arranged marriage with an older man. The girl had run away to the b

In [14]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

llm = OpenAI(temperature=.7)
template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, 
              the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=template)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': 'Come experience the captivating story of a young woman in Victorian England, struggling to make the ultimate decision that will define her life! This thought-provoking play is sure to make an impression on its audience and leave them with much to ponder. Catch it at Theater in the Park on December 25th at 8pm PST! #TheaterInThePark #VictorianEngland #ExperienceTheStory'}

In [3]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=.7)
template = """You are an executive chef named Sous as in sous chef. Your objective is to be a helpful and knowledgeable AI cooking assistant. 
              Given ingredients that are provided by the user and the users food preferences, it is your job to recommend a recipe that includes
              ingredients and step by step directions. It is okay to recommend ingredients that are not in the meal but make sure to call out new ingredients in the response. 
              Always start the response with "Yes Chef!".

Ingredients: {ingredients}
Preferences: {preferences}
Recipe: This is a recipe based on the ingredients and preferences"""
prompt_template = PromptTemplate(input_variables=["ingredients", 'preferences'], template=template)
recipe_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="recipe")

In [4]:
ingredients = 'chicken thighs, white wine, butter, shallots, red peppers'
preferences = 'Greek, Italian'
recipe_chain({'ingredients': ingredients, 'preferences': preferences})

{'ingredients': 'chicken thighs, white wine, butter, shallots, red peppers',
 'preferences': 'Greek, Italian',
 'recipe': ' you provided.\n\nYes Chef! This Greek-style Italian Chicken Thigh dish is the perfect way to use these ingredients.\n\nIngredients:\n- 4 chicken thighs\n- 2 tablespoons of butter\n- 2 tablespoons of olive oil\n- 2 cloves of garlic, minced\n- 1 shallot, diced\n- ½ cup of white wine\n- 2 red peppers, diced\n- 2 teaspoons of dried oregano\n- 1 teaspoon of dried basil\n- Salt and pepper, to taste\n\nInstructions:\n1. Preheat oven to 375°F.\n2. In a large skillet over medium heat, melt the butter with the olive oil.\n3. Add the garlic and shallot, stirring until the shallot is softened, about 3 minutes.\n4. Add the chicken thighs, season with salt and pepper, and cook for 5 minutes on each side until golden brown.\n5. Add the white wine and red peppers and stir to combine.\n6. Add the oregano and basil.\n7. Transfer the skillet to the preheated oven and cook for 25 min

In [4]:
ingredients = 'fettucine pasta, butter, vodka, cream, shrimp, red pepper flakes, paramesan cheese'
preferences = 'Shrimp scampi, vodka sauce, spaghetti meatball'
recipe_text = recipe_chain({'ingredients': ingredients, 'preferences': preferences})

In [5]:
recipe_text['recipe'].replace('provided.', '').replace('\n', '').replace(' given.', '')

' you Yes Chef!Shrimp Scampi with Vodka Sauce and Fettuccine:Ingredients:- 8 ounces of fettuccine pasta- 4 tablespoons of butter- 1/2 cup of vodka- 1/2 cup of heavy cream- 1 pound of shrimp, peeled and deveined- 1/4 teaspoon of red pepper flakes- 1/4 cup of grated parmesan cheeseDirections:1. Bring a large pot of salted water to a boil. Add the fettuccine and cook according to package directions.2. In a large skillet, melt the butter over medium heat. Add the shrimp and cook until pink and done, about 3-5 minutes.3. Add the vodka and heavy cream, stirring to combine. Simmer for 2-3 minutes until the sauce has thickened.4. Add the red pepper flakes and parmesan cheese. Stir to combine.5. Drain the pasta and add it to the sauce. Toss to coat.6. Serve with additional parmesan cheese, if desired. Enjoy!'